In [1]:
#!pip install natasha

In [8]:
import json
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
 
    Doc
)


from yargy.record import Record
from yargy.check import assert_equals

from yargy.record import Record
from yargy.check import assert_type
from yargy.token import (
    join_normalized_tokens,
    join_inflected_tokens
)
import pymorphy2
from razdel import sentenize

from yargy import or_
from yargy.predicates import caseless, normalized, dictionary
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte
from yargy.pipelines import morph_pipeline


In [10]:
!wget "https://www.dropbox.com/s/zjg18xrsobih4ei/merged_data.json"


--2023-01-23 02:06:56--  https://www.dropbox.com/s/zjg18xrsobih4ei/merged_data.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/zjg18xrsobih4ei/merged_data.json [following]
--2023-01-23 02:06:56--  https://www.dropbox.com/s/raw/zjg18xrsobih4ei/merged_data.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc378d816d25f12c341ef725a0ec.dl.dropboxusercontent.com/cd/0/inline/B1F7-FqXbEQBv_Lo8HscSS-LGvzn_9XrhgnTooHVR6le5KCOsym47aCZvmX8Scd1T9G02CtoDtDtHCCZw-D0NEbXh5W4cx1KZZfxGLR4wkoyV9dn1XAHhhR83mDGs5VR-Ea6cger1CSnpJ2dC5x-aIG6pORNbtTmX4VuC1vG-MIYzA/file# [following]
--2023-01-23 02:06:56--  https://uc378d816d25f12c341ef725a0ec.dl.dropboxusercontent.com/cd/0/inline/B1F7-FqXbEQBv_Lo8HscSS-LGvzn_9XrhgnTooHVR6le5KCOsym47aCZvmX8Scd1T9G0

In [11]:
with open('merged_data.json', "r") as readFile:
            data = json.load(readFile, encoding='utf-8')

Вытягиваем все task_text



In [16]:
def task_text(data):
  task_text = []
  for n in range(10000):
      try:
          for i in range(1,20):
              task_text.append(data[n]['Tasks']['Task{}'.format(i)]['task_text'])
      except KeyError:
          continue
      except IndexError:
          continue
  return task_text


text_task = task_text(data)



Вытягиваем задания

In [18]:
def sent(text):
  task = []
  for n in range(len(text)):
    task_all = text[n].split('.')
    task.append(task_all[0])
  
  return task

task = sent(text_task)

In [19]:
len(text_task)

50455

Вытягиваем должности

In [24]:
morph = pymorphy2.MorphAnalyzer()

In [25]:
def task_responsibles_people(data):
  task_res_people = []
  for n in range(10000):
      try:
          for i in range(1,20):
              task_res_people.append(data[n]['Tasks']['Task{}'.format(i)]['task_responsibles_people'])
      except KeyError:
          continue
      except IndexError:
          continue
  return task_res_people


responsible = task_responsibles_people(data)


'Ответственность за исполнение распоряжения возложить на специалиста первой категории административно-хозяйственного отдела Кравцову И.Е.'

In [57]:
def position(text: list) -> list:
  pos = []

  for words in text:
    words = words.split(' ')
    for n in range(len(words)):
      morph_word = morph.parse(words[n])
      if 'anim' in morph_word[0].tag and 'Surn' not in morph_word[0].tag or \
      'ADJF' in morph_word[0].tag and 'gent' in morph_word[0].tag and 'femn' in morph_word[0].tag\
      or 'gent' in morph_word[0].tag and 'NOUN' in morph_word[0].tag and 'masc' not in morph_word[0].tag:  
        pos.append(morph_word[0].word)
    return pos

posis = position(position_words)

In [154]:
NOUN = gram(['NOUN','Sgtm','Surn','anim','datv','masc','sing','accs','femn','inan'])
INFN = gram(['INFN','perf','tran'])
ADJF = gram(['ADJF','Qual','sing','gent','neut','sing','masc','nomn'])
ADVB = gram(['ADVB','Qual','accs','femn','sing'])
PREP = gram('PREP')
CONJ = gram('CONJ')
Surn = gram('Surn')
Name = gram('Name')
Patr = gram('Patr')
ABR = gram('Abbr')



In [276]:
words_position = """Директор предприятия
Финансовый директор (заместитель директора по финансам)
Главный бухгалтер
Главный диспетчер
Главный инженер
Главный конструктор
Главный металлург
Главный метролог
Главный механик
Главный сварщик
Главный специалист по защите информации
Главный технолог
Главный энергетик
Директор (начальник) вычислительного (информационно-вычислительного) центра
Директор гостиницы
Директор котельной
Директор по связям с инвесторами
Директор типографии
Заведующая машинописным бюро
Заведующий архивом
Заведующий бюро пропусков
Заведующий камерой хранения
Заведующий канцелярией
Заведующий копировально-множительным бюро
Заведующий жилым корпусом пансионата (гостиницы)
Заведующий научно-технической библиотекой
Заведующий общежитием
Заведующий производством (шеф-повар)
Заведующий складом
Заведующий столовой
Заведующий фотолабораторией
Заведующий хозяйством
Заведующий экспедицией
Заместитель директора по капитальному строительству
Заместитель директора по коммерческим вопросам
Заместитель директора по связям с общественностью
Заместитель директора по управлению персоналом
Корпоративный секретарь акционерного общества
Мастер контрольный (участка, цеха)
Мастер участка
Менеджер
Менеджер по персоналу
Менеджер по рекламе
Менеджер по связям с инвесторами
Менеджер по связям с общественностью
Начальник автоколонны
Начальник гаража
Начальник (заведующий) мастерской
Начальник инструментального отдела
Начальник исследовательской лаборатории
Начальник производственной лаборатории (по контролю производства)
Начальник лаборатории (бюро) по организации труда и управления производством
Начальник лаборатории (бюро) социологии труда
Начальник лаборатории (бюро) технико-экономических исследований
Начальник нормативно-исследовательской лаборатории по труду
Начальник отдела автоматизации и механизации производственных процессов
Начальник отдела автоматизированной системы управления производством
Начальник отдела информации
Начальник отдела кадров
Начальник отдела капитального строительства
Начальник отдела комплектации оборудования
Начальник отдела контроля качества
Начальник отдела маркетинга
Начальник отдела материально-технического снабжения
Начальник отдела организации и оплаты труда
Начальник отдела охраны окружающей среды
Начальник отдела охраны труда
Начальник отдела патентной и изобретательской работы
Начальник отдела подготовки кадров
Начальник отдела по связям с инвесторами
Начальник отдела (лаборатории, сектора) по защите информации
Начальник отдела по связям с общественностью
Начальник отдела сбыта
Начальник отдела социального развития
Начальник отдела стандартизации
Начальник планово-экономического отдела
Начальник производственного отдела
Начальник ремонтного цеха
Начальник смены
Начальник технического отдела
Начальник финансового отдела
Начальник хозяйственного отдела
Начальник центральной заводской лаборатории
Начальник цеха
Начальник цеха опытного производства
Начальник юридического отдела
Производитель работ (прораб)
Руководитель группы по инвентаризации строений и сооружений
Управляющий отделением
Администратор
Аналитик
Аудитор
Аукционист
Биржевой маклер
Брокер
Брокер торговый
Бухгалтер
Бухгалтер-ревизор
Дилер
Диспетчер
Документовед
Инженер
Инженер-конструктор (конструктор)
Инженер-лаборант
Инженер по автоматизации и механизации производственных процессов
Инженер по автоматизированным системам управления производством
Инженер по защите информации
Инженер по инвентаризации строений и сооружений
Инженер по инструменту
Инженер по качеству
Инженер по комплектации оборудования
Инженер по метрологии
Инженер по надзору за строительством
Инженер по наладке и испытаниям
Инженер по научно-технической информации
Инженер по нормированию труда
Инженер по организации труда
Инженер по организации управления производством
Инженер по охране окружающей среды (эколог)
Инженер по охране труда
Инженер по патентной и изобретательской работе
Инженер по подготовке кадров
Инженер по подготовке производства
Инженер по ремонту
Инженер по стандартизации
Инженер-программист (программист)
Инженер-технолог (технолог)
Инженер-электроник (электроник)
Инженер-энергетик (энергетик)
Инспектор по кадрам
Инспектор по контролю за исполнением поручений
Инспектор фонда
Инструктор-дактилолог
Консультант по налогам и сборам
Лаборант
Математик
Методист по физической культуре
Механик
Оценщик
Переводчик
Переводчик-дактилолог
Переводчик синхронный
Профконсультант
Психолог
Социолог
Специалист по автотехнической экспертизе (эксперт-автотехник)
Специалист по защите информации
Специалист по кадрам
Специалист по маркетингу
Специалист по связям с инвесторами
Специалист по промышленной безопасности подъемных сооружений
Специалист по связям с общественностью
Техник
Техник вычислительного (информационно-вычислительного) центра
Техник-конструктор
Техник-лаборант
Техник по защите информации
Техник по инвентаризации строений и сооружений
Техник по инструменту
Техник по метрологии
Техник по наладке и испытаниям
Техник по планированию
Техник по стандартизации
Техник по труду
Техник-программист
Техник-технолог
Товаровед
Физиолог
Художник
Художник-конструктор (дизайнер)
Шеф-инженер
Экономист
Экономист вычислительного (информационно-вычислительного) центра
Экономист по бухгалтерскому учету и анализу хозяйственной деятельности
Экономист по договорной и претензионной работе
Экономист по материально-техническому снабжению
Экономист по планированию
Экономист по сбыту
Экономист по труду
Экономист по финансовой работе
Эксперт
Эксперт дорожного хозяйства
Эксперт по промышленной безопасности подъемных сооружений
Юрисконсульт
Агент
Агент коммерческий
Агент по закупкам
Агент по продаже недвижимости
Агент по снабжению
Агент рекламный
Агент страховой
Агент торговый
Архивариус
Ассистент инспектора фонда
Дежурный бюро пропусков
Дежурный (по выдаче справок, залу, этажу гостиницы, комнате отдыха, общежитию и др.)
Делопроизводитель
Инкассатор
Калькулятор
Кассир
Кодификатор
Комендант
Коммивояжер
Копировщик
Крупье
Машинистка
Нарядчик
Оператор диспетчерской движения и погрузочно-разгрузочных работ
Оператор диспетчерской службы
Оператор по диспетчерскому обслуживанию лифтов
Секретарь-машинистка
Секретарь незрячего специалиста
Секретарь руководителя
Секретарь-стенографистка
Статистик
Стенографистка
Табельщик
Таксировщик
Учетчик
Хронометражист
Чертежник
Экспедитор
Экспедитор по перевозке грузов
Директор (начальник) учреждения (организации)
Заместитель директора (начальника) учреждения (организации) по научной работе. Главный инженер учреждения (организации)
Ученый секретарь
Заведующий (начальник) научно-исследовательским отделом (отделением, лабораторией) института
Заведующий (начальник) научно-исследовательским отделом (лабораторией) учреждения; заведующий (начальник) научно-исследовательским сектором (лабораторией), входящим в состав научно-исследовательского отдела (отделения, лаборатории) института
Главный научный сотрудник
Ведущий научный сотрудник
Старший научный сотрудник
Научный сотрудник
Младший научный сотрудник
Заведующий аспирантурой
Заведующий (начальник) отделом стандартизации
Заведующий (начальник) отделом научно-технической информации
Заведующий (начальник) планово-экономическим отделом
Заведующий (начальник) отделом кадров
Заведующий техническим архивом
Заведующий фотолабораторией
Ведущий инженер
Ведущий экономист
Инженер
Экономист
Переводчик
Художник
Техник
Лаборант
Главный конструктор проекта
Главный инженер проекта. Главный архитектор проекта
Главный ландшафтный архитектор проекта
Заведующий конструкторским отделом
Заведующий отделом (бюро) оформления проектных материалов
Заведующий чертежно-копировальным бюро
Начальник (руководитель) бригады (группы)
Главный специалист в основном отделе (архитектурно-планировочной мастерской)
Ведущий конструктор
Инженер-проектировщик
Архитектор
Ландшафтный архитектор
Техник-проектировщик
Чертежник-конструктор
Руководитель подразделения
Главный редактор
Научный редактор
Редактор
Технический редактор
Художественный редактор
Выпускающий
Младший редактор
Корректор

"""


In [277]:
words_position = words_position.split()


for n in range(len(words_position)):
  morph_word_2 = morph.parse(words_position[n])
  if 'PREP' in morph_word_2[0].tag or 'CONJ' in morph_word_2[0].tag or 'PRCL' in morph_word_2[0].tag:
    morph_word[0].tag
  words_position[n] = words_position[n].replace(morph_word_2[0].word, '')


words_position = set(words_position)
words_position = list(words_position)
words_position = words_position[1:]

In [282]:
from yargy import Parser
from yargy.pipelines import morph_pipeline
from yargy.predicates import eq, type as _t, normalized, gram
from ipymarkup import show_span_box_markup, show_span_line_markup
from yargy.tokenizer import MorphTokenizer


POSITION = morph_pipeline(words_position)

parser = Parser(POSITION)


In [312]:
text = responsible

def position_words(text):
  words_position = []
  for n in range(len(text)):
    match = list(parser.findall(text[n]))
    for value in match:
      words_position.append(value.tokens[0].value)
  return words_position

words_position_word = position_words(text)

In [313]:
words_position_word

['специалиста',
 'специалиста',
 'администратора',
 'специалистом',
 'технического',
 'специалистом',
 'специалиста',
 'специалиста',
 'специалиста',
 'специалиста',
 'специалиста',
 'руководителя',
 'специалистом',
 'специалиста',
 'руководителя',
 'руководителя',
 'специалиста',
 'специалиста',
 'специалиста',
 'главного',
 'бухгалтера',
 'специалиста',
 'специалиста',
 'администратора',
 'администратора',
 'специалиста',
 'специалистом',
 'руководителя',
 'руководителем',
 'специалиста',
 'специалиста',
 'руководителя',
 'специалиста',
 'руководителя',
 'специалиста',
 'специалиста',
 'специалиста',
 'архитектором',
 'специалиста',
 'архитектора',
 'специалиста',
 'специалиста',
 'специалиста',
 'архитектором',
 'специалиста',
 'специалиста',
 'специалиста',
 'специалиста',
 'специалиста',
 'специалиста',
 'администратора',
 'руководителя',
 'руководителя',
 'специалиста',
 'руководителя',
 'специалиста',
 'специалиста',
 'специалиста',
 'главного',
 'бухгалтера',
 'специалиста',
 '

# Вытаскиваем ФИО

In [ ]:
def fio(text):
  fio = []
  for n in range(len(text)):
    morph_word = morph.parse(text[n])
    if 'anim' in morph_word[0].tag and 'Name' in morph_word[0].tag or 'Surn' in morph_word[0].tag \
    or 'UNKN' in morph_word[0].tag:
      fio.append(morph_word[0][2])
  return fio


In [ ]:
text_abbr = fio(text_w)

# Вытаскиваем ФИО с аббривиатурой

In [ ]:
def fio_abbr(text):
  new_fio_abbr = []
  for n in range(len(text_abbr)):
    if '.' in text_w[n]:
      new_fio_abbr.append(text_w[n-1])
      new_fio_abbr.append(text_w[n])
  return new_fio_abbr



In [ ]:
fio = []
for word in text:
    word = morph.parse(word)
   
    if 'Surn' in word[0][1] or 'Name' in word[0][1] or 'Patr' in word[0][1]:
        fio.append(word[0][2])
        
fio

['ответственность за исполнение распоряжения возложить на специалиста третьей категории службы ит кима юрия иванович',
 'контроль над выполнением приказа возложить на системного администратора сергеева андрей']

In [ ]:
from natasha import NamesExtractor
from natasha.markup import show_markup, show_json

extractor = NamesExtractor()

text = '''
Благодарственное письмо   Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника:  Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. Они вовлекали сына в интересные внеурочные занятия, связанные с театром и походами.

Благодарю прекрасного учителя 1"А" класса - Волкову Наталью Николаевну, нашего наставника, тьютора - Ларису Ивановну, за огромнейший труд, чуткое отношение к детям, взаимопонимание! Огромное спасибо!
'''
matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
show_json(facts)